# Recommenders with NMF

**Non-Negative Matrix Factorization (NMF)**

In [1]:
import numpy as np
from sklearn.decomposition import NMF
import pandas as pd

In [2]:
movies = ['Titanic', 'Breakfast at Tiffanys', 'Terminator', 'Star Trek', 'Star Wars']
users = ['Ada', 'Bob', 'Charlie', 'Steve']
Rtrue = np.array([[5, 4, 3, 2, 1],
                  [3, 2, 0, 0, 1],
                  [0, 0, 0, 0, 5],
                  [1, 0, 4, 5, 5]])

In [3]:
df = pd.DataFrame(Rtrue, index=users, columns=movies)
df

,Titanic,Breakfast at Tiffanys,Terminator,Star Trek,Star Wars
Ada,5,4,3,2,1
Bob,3,2,0,0,1
Charlie,0,0,0,0,5
Steve,1,0,4,5,5


#### Model parameters
(first manually)

In [4]:
# mapping of 2 genres: Drama+Scifi to our 5 movies
P = np.array([[3, 2, 3, 1, 1],    # drama
              [1, 0, 2, 3, 3]])   # scifi

In [5]:
# mapping of 2 genres to our 4 users
Q = np.array([[3, 2, 0, 0.1],   # drama
              [1, 0, 3, 3]])  # scifi

In [6]:
P.shape, Q.shape   # --> (4, 5)

((2, 5), (2, 4))

In [7]:
# how the dot product works, one line example
ada = (3*3 + 1*1), (3*2 + 1*0), (3*3 + 1*2), (3*1 + 1*3), (3*1 + 1*3)

In [8]:
Q.T

array([[3. , 1. ],
       [2. , 0. ],
       [0. , 3. ],
       [0.1, 3. ]])

In [9]:
P

array([[3, 2, 3, 1, 1],
       [1, 0, 2, 3, 3]])

In [10]:
R = (np.dot(Q.T, P) * 0.5).round().astype(int)
R

array([[5, 3, 6, 3, 3],
       [3, 2, 3, 1, 1],
       [2, 0, 3, 4, 4],
       [2, 0, 3, 5, 5]])

$R \sim Q^T \cdot P$

We want to choose the values in Q and P in such a way that the reconstructed matrix R
becomes as similar as possible to our observed data $R_{true}$

quadratic_loss = $\sum(R-R_{true})^2$

we can optimize the quadratic loss using Gradient Descent

In [11]:
np.sum((R - Rtrue)**2)

57

### Train the model

In [12]:
from sklearn.decomposition import NMF

m = NMF(n_components=2)   # <-- the other dimension of P and Q; how many "genres"
m.fit(Rtrue)

NMF(n_components=2)

In [13]:
P = m.components_
P

array([[0.01956503, 0.        , 1.2434145 , 1.61558857, 2.49980691],
       [2.07380417, 1.559131  , 0.78959006, 0.42075471, 0.        ]])

In [14]:
Q = m.transform(Rtrue)
Q

array([[0.50703494, 2.51628848],
       [0.04469421, 1.22989243],
       [1.20118075, 0.        ],
       [2.38623553, 0.43471077]])

In [15]:
np.dot(Q, P).round()

array([[5., 4., 3., 2., 1.],
       [3., 2., 1., 1., 0.],
       [0., 0., 1., 2., 3.],
       [1., 1., 3., 4., 6.]])

### Getting recommendations for new users

In [16]:
user = np.array([[0, 5, 1, 0, 0]])   # user likes Tiffany but not Terminator

In [17]:
profile = m.transform(user)          # how strongly our user likes the 2 "genres"
profile

array([[0.        , 1.13982996]])

In [21]:
profile.shape

(1, 2)

In [22]:
P.shape

(2, 5)

In [18]:
result = np.dot(profile, P)          # how strongly our user would like all 5 movies
result

array([[2.36378412, 1.77714422, 0.8999984 , 0.47958882, 0.        ]])

In [19]:
s = pd.Series(result[0], index=movies)
s = s.drop(['Terminator', 'Breakfast at Tiffanys'])
s.sort_values(ascending=False).head(3)

Titanic      2.363784
Star Trek    0.479589
Star Wars    0.000000
dtype: float64

### Practical Hints

1. create a matrix for a new user (1 x n_movies)
2. insert a few movie ratings (try an extreme taste first)
3. fill up the rest with zeroes
4. (optional) replace zeros with something else:
   * 2.5 (Amirali)
   * mean of all nonzero values (standard)
   * 0, ignore (probably bad)
   * 3 (OK)
   * try the Surprise library
5. transform the new user with the NMF -> profile (1 x N)
6. inspect the shape of the profile
7. multiply the profile with P (dot product)
8. inspect the shape of the result
9. put the movie name labels back (pd.Series)
10. remove all movies the new user has already seen
11. sort in descending order, recommend Top 5

$\approx$